In [358]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from tqdm.notebook import tqdm

import os

path = "../Dataset/API Datasets/"
file_list = os.listdir(path)
file_list_py = [file for file in file_list if file.endswith('.csv')]

book = pd.read_csv("../Dataset/items.csv", sep="|")

In [359]:
df = pd.DataFrame()
for i in tqdm(file_list_py):
    data = pd.read_csv(path + i, sep="|", encoding='utf-16', error_bad_lines=False)
    df = pd.concat([df,data])
    
df = df.reset_index(drop = True)
apidata = df

In [360]:
apidata.head(3)

,titleQueried,titleFound,author,language,description,isbn
0,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.780553e+12
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12
2,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12


In [361]:
book.head(3)

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"


In [362]:
apidata.sort_values('titleQueried')
apidata.head(3)

,titleQueried,titleFound,author,language,description,isbn
0,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.780553e+12
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12
2,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12


In [363]:
apidata.isnull().sum()

titleQueried         0
titleFound       11734
author          113304
language         12113
description     314420
isbn            259934
dtype: int64

In [364]:
apidata.shape

(741289, 6)

In [365]:
# remove row with no author, no title Found
# processedApi = apidata[apidata['author'].notna()]
# processedApi = processedApi[processedApi['titleFound'].notna()]

# fill na values into '' string
processedApi = apidata.copy()
processedApi['author'] = processedApi['author'].fillna('')
processedApi['titleFound'] = processedApi['titleFound'].fillna('')

In [366]:
processedApi.isnull().sum()

titleQueried         0
titleFound           0
author               0
language         12113
description     314420
isbn            259934
dtype: int64

In [367]:
processedApi.head(3)

,titleQueried,titleFound,author,language,description,isbn
0,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.780553e+12
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12
2,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9.781409e+12


In [368]:
# make a dataframe w/ books in common between original data and apidata
mask = processedApi['titleQueried'].isin(book['title'])
booksInCommon,processedApi_left = processedApi[mask],processedApi[~mask]

In [369]:
processedApi_left

,titleQueried,titleFound,author,language,description,isbn
12,Einfach zeichnen! Step by Step|Business-Symbol...,,,NaN,NaN,NaN
23,Red Queen 1|Gläsernes Schwert (Die Farben des ...,,,NaN,NaN,NaN
25,Red Queen 1|Zerschlagene Krone - Geschichten u...,,,NaN,NaN,NaN
26,Red Queen 1|A thousand boy kisses|Tillie Cole|...,,,NaN,NaN,NaN
94,Pedro und die Bettler von Cartagena|Kurvenwass...,,,NaN,NaN,NaN
...,...,...,...,...,...,...
152703,The Pit and the Pendulum|The Assignation/The P...,,,NaN,NaN,NaN
152727,Der dunkle Vampir|Vampire - der dunkle Mythos|...,,,NaN,NaN,NaN
152737,The Greatest Demon Lord Is Reborn as a Typical...,,,NaN,NaN,NaN
152741,The Greatest Demon Lord Is Reborn as a Typical...,,,NaN,NaN,NaN


In [370]:
booksInCommon = booksInCommon.sort_values('titleQueried')
booksInCommon['key:title-author'] = ((booksInCommon['titleFound'] + ' '+ booksInCommon['author']).astype(str)).str.lower()
booksInCommon['key:title-author'] = booksInCommon['key:title-author'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
# booksInCommon = booksInCommon.drop_duplicates('key:title-author', keep='first')
booksInCommon.reset_index(inplace=True)
booksInCommon.drop(['index'], axis=1, inplace=True)
booksInCommon.head(3)

,titleQueried,titleFound,author,language,description,isbn,key:title-author
0,TRIPION minaccia dallo spazio,Allgemeine Naturgeschichte und Theorie Des Him...,Immanuel Kant,de,Dieses Werk ist Teil der Buchreihe TREDITION C...,9.783842e+12,allgemeinenaturgeschichteundtheoriedeshimmelsi...
1,TRIPION minaccia dallo spazio,Jean-Luc Godard,Roberto Chiesi,it,Filmographie (p. 101-116).,9.788884e+12,jeanlucgodardrobertochiesi
2,TRIPION minaccia dallo spazio,Cineforum,,it,NaN,NaN,cineforum


In [371]:
booksInCommon[booksInCommon['titleQueried'] == 'Zwischen Sucht und Liebe']

,titleQueried,titleFound,author,language,description,isbn,key:title-author
727534,Zwischen Sucht und Liebe,"General Moorner, oder der Streit zwischen Lieb...",Friedrich Theophilus Thilo,de,NaN,NaN,generalmoorneroderderstreitzwischenliebeundpfl...
727535,Zwischen Sucht und Liebe,"J. M. Sailer's sämmtliche Werke, unter Anleitu...",Johann Michael Sailer,de,NaN,NaN,jmsailerssämmtlichewerkeunteranleitungdesverfa...
727536,Zwischen Sucht und Liebe,Beziehungssucht und Co-Abhängigkeit bei Frauen,Sabine Schmidt,de,Wird die Abhängigkeit eines Menschen offensich...,9.783958e+12,beziehungssuchtundcoabhängigkeitbeifrauensabin...
727537,Zwischen Sucht und Liebe,Sucht ohne Drogen,Werner Gross,de,NaN,NaN,suchtohnedrogenwernergross
727538,Zwischen Sucht und Liebe,“Die” vom Himmel geseegnete Liebe zwischen Isa...,,de,NaN,NaN,dievomhimmelgeseegneteliebezwischenisaacundreb...
727539,Zwischen Sucht und Liebe,Wolf,Tamara Ross,en,NaN,9.783746e+12,wolftamaraross
727540,Zwischen Sucht und Liebe,Evangelien-Postille für die Sonn- und Festtage...,Wilhelm Löhe,de,NaN,NaN,evangelienpostillefürdiesonnundfesttagedeskirc...
727541,Zwischen Sucht und Liebe,Wolf,Tamara Ross,en,NaN,9.783746e+12,wolftamaraross
727542,Zwischen Sucht und Liebe,Hexe,Tamara Ross,de,"Ein Kind, ein Haus in Rumänien und der richtig...",9.783748e+12,hexetamaraross
727543,Zwischen Sucht und Liebe,Zwischen Sucht und Liebe,Tamara Ross,de,Shiva ist eine Vampwitch: Halb Hexe und halb V...,9.783739e+12,zwischensuchtundliebetamaraross


In [372]:
book.isnull().sum()

itemID           0
title            0
author        3240
publisher        9
main topic     258
subtopics        1
dtype: int64

In [373]:
book['author'] = book['author'].fillna('')
book['key:title-author'] = ((book['title'] + ' '+ book['author']).astype(str)).str.lower()
book['key:title-author'] = book['key:title-author'].str.replace(pat=r'[^\w]', repl=r'', regex=True)
book = book.sort_values('title')
book.reset_index(inplace=True)
book.drop(['index'], axis=1, inplace=True)
book.head(3)

,itemID,title,author,publisher,main topic,subtopics,key:title-author
0,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],tripionminacciadallospaziostefanogrimaldi
1,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],theultimatevehiclecolouringbookforkidschetna
2,64846,!!,,AUTHORHOUSE,FM,[],


In [374]:
#book[book['title'] == '!!']

In [375]:
# check similarity
dftemp = DataFrame(range(0),columns=list(booksInCommon.columns))
dftemp

,titleQueried,titleFound,author,language,description,isbn,key:title-author


In [376]:
# check similarity
dftemp = DataFrame(range(0),columns=list(booksInCommon.columns))
threshold = 70
threshold2 = 50  ## Conservative standards to drop unrelated apidata
j_end = 0

for i in tqdm(range(len(book))):    
    for j in range(j_end,len(booksInCommon)):
        # if titles from two data are the same
        if book['title'].iloc[i] == booksInCommon.get('titleQueried').iloc[j]:
            j_end += 1
            # if similarity between key:title-author > threshold
            if (fuzz.token_sort_ratio(booksInCommon.get('key:title-author').iloc[j], book.get('key:title-author').iloc[i])) > threshold:
                # if similarity between titleQueried and titleFound  > threshold2, then save it as a dataframe
                if (fuzz.token_sort_ratio(booksInCommon.get('titleQueried').iloc[j], booksInCommon.get('titleFound').iloc[j])) > threshold2:
                    dftemp = dftemp.append(booksInCommon.iloc[j])     
        else:
            j_end = j
            break;

In [377]:
dftemp.reset_index(inplace=True)
dftemp.drop(['index'], axis=1, inplace=True)
display(dftemp.head(5))

,titleQueried,titleFound,author,language,description,isbn,key:title-author
0,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,it,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,9.781292e+12,tripionminacciadallospaziostefanogrimaldi
1,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,en,NaN,9.789390e+12,theultimatevehiclecolouringbookforkidschetna
2,!!,,,NaN,NaN,NaN,
3,!!,,,NaN,NaN,NaN,
4,!!,,,NaN,NaN,NaN,


In [378]:
a = '#4 Happy Birthday, Mallory!'
b = 'Happy Birthday, Mallory!'
a_ = '4HappyBirthdayMallory'
b_ = 'HappyBirthdayMallory'
print(fuzz.token_sort_ratio(a, b))
print(fuzz.token_sort_ratio(a_, b_))

96
98


In [379]:
# check whether they are from collection, the same book, or different book
n = 8

dftemp2 = dftemp.copy()
dftemp2['titleQueried'] = (dftemp2['titleQueried']).astype(str).str.lower()
dftemp2['titleFound'] =(dftemp2['titleFound']).astype(str).str.lower()
dftemp2["titleQueried"] = dftemp2["titleQueried"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
dftemp2["titleFound"] = dftemp2["titleFound"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

diffList = []
sameList = []
collectionList = []

for i in tqdm(range(len(dftemp2))):
    # if title is not the same --> need to further check
    if dftemp2.get('titleQueried').iloc[i] != dftemp2.get('titleFound').iloc[i]:
        ## if titleQueried and titleFound similarity < 97 --> need to further check 
        if fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i], dftemp2.get('titleFound').iloc[i]) < 97:
            ### if firt n-letter similarity > 80 --> they are from a collection
            if (fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i][:n], dftemp2.get('titleFound').iloc[i][:n]) > 80):
                collectionList.append(dftemp.iloc[i])
            elif fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i], dftemp2.get('titleFound').iloc[i]) > 70:
                collectionList.append(dftemp.iloc[i])
            else:
                diffList.append(dftemp.iloc[i])
        ## if titleQueried and titleFound similarity > 97, they are the same book 
        else:
            sameList.append(dftemp.iloc[i])
    # if titme is the same, they are the same book
    else:
        sameList.append(dftemp.iloc[i])
            
collectionDf = pd.DataFrame(collectionList)
diffDf = pd.DataFrame(diffList)
sameDf = pd.DataFrame(sameList)

In [380]:
#fuzz.token_sort_ratio('Alex Rider 2: Stormbreaker', 'Alex Rider 1: Stormbreaker')

### collection: collection(1) same book(2)

In [381]:
# collectionDf, diffDf, sameDf
diffDf.to_excel('diffDf.xlsx')
diffDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author
8,"#4 Happy Birthday, Mallory!",#06 Heart to Heart with Mallory,Laurie Friedman,en,Valentine's Day is approaching and everyone se...,9.780823e+12,06hearttoheartwithmallorylauriefriedman
54,'n Gotiese Kers Engel (Afrikaanse uitgawe),Put van Drome (Afrikaanse uitgawe),Anna Erishkigal,af,Al wat Rosie Xalbadore wil hê is ‘n werk vir d...,9.781950e+12,putvandromeafrikaanseuitgaweannaerishkigal
56,'n Gotiese Kers Engel (Afrikaanse uitgawe),Die Horlosiemaker: 'n Novelle (Afrikaanse Taal),Anna Erishkigal,af,--Vra hoe jy ‘n uur in tyd kan wen-- . Marae O...,9.781943e+12,diehorlosiemakernnovelleafrikaansetaalannaeris...
63,"'quién Viene a Cenar, Pequeño Hoo?","Who's Coming for Dinner, Little Hoo? / ¿Quién ...",Brenda Ponnay,en,Whooo's There? It's time for Thanksgiving dinn...,9.781624e+12,whoscomingfordinnerlittlehooquiénvieneacenarpe...
71,(Un)arranged Marriage,"BALI RAI '(UN)ARRANGED MARRIAGE', TEACHER'S GUIDE",Bali Rai,en,NaN,9.783126e+12,baliraiunarrangedmarriageteachersguidebalirai
...,...,...,...,...,...,...,...
99527,"»Super«, schreit der Frieder, und die Oma kich...",Und wieder schreit der Frieder Oma,Gudrun Mebs,de,Der zweite Teil der Frieder-Abenteuer! Der fre...,9.783734e+12,undwiederschreitderfriederomagudrunmebs
99542,Ætherresonanz,Aetherhertz,Anja Bagus,de,Ein Annabelle Rosenherz Roman:Seit der Jahrhun...,9.781485e+12,aetherhertzanjabagus
99565,Übernatürliche Irreführung (Sasha Urban Serie 5),"Rauch, Vampire und Spiegel (Sasha Urban Serie:...","Dima Zales, Anna Zaires",de,Das epische Ende der Geschichte von Sasha Urba...,9.781631e+12,rauchvampireundspiegelsashaurbanseriebuch7dima...
99566,Übernatürliche Irreführung (Sasha Urban Serie 5),Widerspenstiges Medium (Sasha Urban Serie: Buc...,"Dima Zales, Anna Zaires",de,Das Leben als Seherin ist auch nicht das Gelbe...,9.781631e+12,widerspenstigesmediumsashaurbanseriebuch3dimaz...


In [382]:
collectionDf['collection'] = 1
collectionDf
collectionDf.to_excel('collectionDf.xlsx')

In [383]:
sameDf['collection'] = 2
sameDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author,collection
0,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,it,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,9.781292e+12,tripionminacciadallospaziostefanogrimaldi,2
1,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,en,NaN,9.789390e+12,theultimatevehiclecolouringbookforkidschetna,2
2,!!,,,NaN,NaN,NaN,,2
3,!!,,,NaN,NaN,NaN,,2
4,!!,,,NaN,NaN,NaN,,2
...,...,...,...,...,...,...,...,...
99563,Überlebensprogramm,Überlebensprogramm,Dieter Rieken,de,In nicht allzu ferner Zukunft: Lisa Ressler ar...,9.783748e+12,überlebensprogrammdieterrieken,2
99564,Übernachtungs-Party,Übernachtungs-Party,,de,NaN,9.783831e+12,übernachtungsparty,2
99569,Überschneidung zweier untersschiedlicher Welten,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,de,Wenn du nach getaner Arbeit nachhause kommst u...,9.783746e+12,überschneidungzweieruntersschiedlicherweltenwe...,2
99570,Übersinnlicher Retter,Übersinnlicher Retter,Tina Folsom,de,"Als eine Hellseherin Pearce warnt, dass er von...",9.781945e+12,übersinnlicherrettertinafolsom,2


### need to go over diffDF/collectionDF mannually --> add to  sameDf / collectionDf if necessary

In [384]:
# mergedDf (sameDf + collectionDf)
merged = pd.concat([sameDf, collectionDf])
merged.sort_values('titleQueried')
merged.rename(columns = {'titleQueried':'title'}, inplace=True)
merged

,title,titleFound,author,language,description,isbn,key:title-author,collection
0,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,it,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,9.781292e+12,tripionminacciadallospaziostefanogrimaldi,2
1,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,en,NaN,9.789390e+12,theultimatevehiclecolouringbookforkidschetna,2
2,!!,,,NaN,NaN,NaN,,2
3,!!,,,NaN,NaN,NaN,,2
4,!!,,,NaN,NaN,NaN,,2
...,...,...,...,...,...,...,...,...
99541,Ætherresonanz,Aetherresonanz,Anja Bagus,de,Baden-Baden 1912: Das Amt für Ætherangelegenhe...,9.783848e+12,aetherresonanzanjabagus,1
99551,Österreichische Volksmärchen,Oesterreichische Volksmärchen,Franz Tschischka,de,NaN,NaN,oesterreichischevolksmärchenfranztschischka,1
99562,Überlebensprogramm,Das Überlebensprogramm,Frederic Vester,de,NaN,NaN,dasüberlebensprogrammfredericvester,1
99568,Übernatürliche Irreführung (Sasha Urban Serie 5),Übernatürliche Irreführung (Sasha Urban Serie:...,"Dima Zales, Anna Zaires",de,Da ich meine Seherkräfte endlich unter Kontrol...,9.781631e+12,übernatürlicheirreführungsashaurbanseriebuch5d...,1


In [385]:
# add api data to items.csv data
addedData = pd.merge(book, merged, how="outer", on="title")
addedData.drop(['key:title-author_x', 'key:title-author_y'], axis=1, inplace=True)
addedData

,itemID,title,author_x,publisher,main topic,subtopics,titleFound,author_y,language,description,isbn,collection
0,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,it,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,9.781292e+12,2.0
1,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,en,NaN,9.789390e+12,2.0
2,64846,!!,,AUTHORHOUSE,FM,[],,,NaN,NaN,NaN,2.0
3,64846,!!,,AUTHORHOUSE,FM,[],,,NaN,NaN,NaN,2.0
4,64846,!!,,AUTHORHOUSE,FM,[],,,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
158297,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),"Dima Zales, Anna Zaires",MOZAIKA LLC,FMX,[],Übernatürliche Irreführung (Sasha Urban Serie:...,"Dima Zales, Anna Zaires",de,Da ich meine Seherkräfte endlich unter Kontrol...,9.781631e+12,1.0
158298,35275,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,Books on Demand,FD,"[FD,FM]",Überschneidung zweier untersschiedlicher Welten,WErner Röschl,de,Wenn du nach getaner Arbeit nachhause kommst u...,9.783746e+12,2.0
158299,50552,Übersinnlicher Retter,Tina Folsom,Duboce Park Press,FRT,[],Übersinnlicher Retter,Tina Folsom,de,"Als eine Hellseherin Pearce warnt, dass er von...",9.781945e+12,2.0
158300,5665,Übungsheft Handlettering: Übungsbuch Für Hand ...,"Sunshine Handlettering, Handlettering Heft In ...",INDEPENDENTLY PUBLISHED,YNV,[],Übungsheft Handlettering: Übungsbuch Für Hand ...,Sunshine Handlettering,de,Blanko Übungsheft für Handschriften und Handle...,9.781092e+12,1.0


In [386]:
apidata[apidata['titleQueried'] == '#Basteln for Future']

,titleQueried,titleFound,author,language,description,isbn
27927,#Basteln for Future,#Basteln for Future,Susanne Pypke,de,Bastle die Welt besser! Der Klimawandel geht u...,9.783736e+12
27929,#Basteln for Future,Professionelles Eurex-Trading,"Christian Eck, Matthias S. Riechert",de,NaN,9.783899e+12
27930,#Basteln for Future,"Wir retten die Bienen, Igel und Käfer!",Susanne Pypke,de,Naturentdecker aufgepasst! Dieses Buch steckt ...,9.783736e+12
27932,#Basteln for Future,Mein Adventskalender-Buch: Christmas for Futur...,NaN,de,NaN,9.783746e+12
27933,#Basteln for Future,Mein Adventskalender-Buch: Girls for Future,Annette Neubauer,de,NaN,9.783746e+12
27934,#Basteln for Future,Tagebuch einer musikalischen Reise,Charles Burney,de,Charles Burney: Tagebuch einer musikalischen R...,NaN
27935,#Basteln for Future,Basteln mit den Allerkleinsten Naturmaterialien,Susanne Pypke,de,Raus in die Natur! Mit diesem Buch und den sel...,9.783736e+12
27936,#Basteln for Future,Alles ist zum Basteln da - Weihnachten,Pia Pedevilla,de,NaN,9.783772e+12


In [387]:
addedData.to_csv('../Dataset/addedData.csv')

In [388]:
# joinedData = pd.merge(book, merged, how="inner", on="title")
# joinedData.drop(['key:title-author_x', 'key:title-author_y'], axis=1, inplace=True)
# joinedData
# joinedData.to_csv('../Dataset/joinedData.csv')

# Preprocessing

In [389]:
addedData = pd.read_csv("../Dataset/addedData.csv")
addedData.head(3)

,Unnamed: 0,itemID,title,author_x,publisher,main topic,subtopics,titleFound,author_y,language,description,isbn,collection
0,0,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,it,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,9.781292e+12,2.0
1,1,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,en,NaN,9.789390e+12,2.0
2,2,64846,!!,NaN,AUTHORHOUSE,FM,[],NaN,NaN,NaN,NaN,NaN,2.0


In [390]:
prep = addedData.drop_duplicates('itemID', keep='first').sort_values('title')
prep.reset_index(inplace=True)
prep.drop(['Unnamed: 0'], axis=1, inplace=True)
prep.drop(['index'], axis=1, inplace=True)
prep = prep[["itemID", "title", "titleFound", "author_x", "author_y", "language", "collection", "isbn", "publisher", "description", "main topic", "subtopics"]]
prep["author_y"] = prep["author_y"].str.replace(pat=';.*', repl=r'', regex=True)


In [391]:
print(prep.shape)
display(prep.head(3))

(78030, 12)


,itemID,title,titleFound,author_x,author_y,language,collection,isbn,publisher,description,main topic,subtopics
0,26966,TRIPION minaccia dallo spazio,""" TRIPION "" minaccia dallo spazio",Stefano Grimaldi,Stefano Grimaldi,it,2.0,9.781292e+12,Lulu.com,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,[]
1,24125,The Ultimate Vehicle Colouring Book for Kids,""" The Ultimate Vehicle Colouring Book for Kids""",Chetna .,Chetna .,en,2.0,9.789390e+12,Westland Publications Limited,NaN,YBG,[]
2,64846,!!,NaN,NaN,NaN,NaN,2.0,NaN,AUTHORHOUSE,NaN,FM,[]


In [392]:
prep.isnull().sum()

itemID             0
title              0
titleFound     21598
author_x        3240
author_y       23354
language       21601
collection     21597
isbn           23838
publisher          9
description    29643
main topic       258
subtopics          1
dtype: int64

In [393]:
import re


prep['author_x'] = prep['author_x'].fillna('')
prep['author_y'] = prep['author_y'].fillna('')

string1 = ((prep['title'] + ' '+ prep['author_x']).astype(str)).str.lower()
string2 = ((prep['title'] + ' '+ prep['author_y']).astype(str)).str.lower()

# string1.iloc[3]

for i in tqdm(range(len(prep))):
    prep.at[i, 'key1'] = (re.sub(r"[^a-zA-Z0-9]","", string1.iloc[i]))
    prep.at[i, 'key2'] = (re.sub(r"[^a-zA-Z0-9]","", string2.iloc[i]))

prep['itemID'] = prep['itemID'].astype(str)
prep['main topic'] = prep['main topic'].astype(str)

In [394]:
# prepocessing of collection == 1

col_1 = prep[prep['collection'] == 1]
keys = prep['key1']
col_11 = col_1[keys.isin(keys[keys.duplicated()])]

col_11 = col_11.groupby('key1').agg({'itemID': '|'.join,
                                    'title': 'first', 
                                    'author_x': 'first',
                                    'language':'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher':'first',
                                    'description':'first',
                                    'main topic': '|'.join,
                                    'subtopics': '|'.join }).reset_index()

col_11.columns =['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']


subtopics = col_11['subtopics']

for i in range(len(col_11)):
#     col_11['subtopics'][i] = col_11['subtopics'][i].replace('[]', '')
    subtopics.at[i] = re.sub("\[|\]|\'", "", str(subtopics[i]))
    
    subtopics.at[i] = re.sub(",|, ", "|", str(subtopics.at[i]))
    subtopics.at[i] = str(subtopics.at[i].strip('|'))
    if subtopics.at[i] == 'nan':
        subtopics.at[i] = ''        
    
col_11['subtopics'] = subtopics
    
idTemp = np.array(col_11["itemID"].tolist())
mainTopicTemp = np.array(col_11["main topic"].tolist())
subtopicTemp = np.array(col_11["subtopics"].tolist())
idList = []
mainTopicList = []
subtopicList = []

for i in range(len(col_11)):
    arrTemp1 = idTemp[i].split('|')
    arrTemp2 = mainTopicTemp[i].split('|')
    arrTemp3 = subtopicTemp[i].split('|')
    
    idList.append(arrTemp1)
    mainTopicList.append(arrTemp2)
    subtopicList.append(arrTemp3)
    
# remove duplication 
for i in range(len(col_11)):
    idList[i] = list(set(idList[i]))
    mainTopicList[i] = list(set(mainTopicList[i]))
    subtopicList[i] = list(set(subtopicList[i]))
    
col_11['itemID'] = (idList)
col_11['main topic'] = (mainTopicList)
col_11['subtopics'] = (subtopicList)

# subtract col_11 from col_1 
for i in range(len(col_11)):
    for j in range(len(col_11['itemID'][i])):
        subtId = str(int(col_1[col_1['itemID'].isin([col_11['itemID'][i][j]])]['itemID']))
        idx = col_1[col_1['itemID'] == subtId].index
        col_1 = col_1.drop(idx)


col_1.rename(columns = {'key1':'key'}, inplace=True)
col_1.rename(columns = {'author_x':'author'}, inplace=True)
col_1.drop(['titleFound', 'author_y', 'key2'], axis=1, inplace=True)
col_1 = col_1[['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']]

newCol1 = pd.concat([col_1, col_11])
print(newCol1.shape)
display(newCol1.head(5))

<ipython-input-394-3a083dab0c4a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  col_11 = col_1[keys.isin(keys[keys.duplicated()])]


(7410, 11)


,key,itemID,title,author,language,collection,isbn,publisher,description,main topic,subtopics
11,cincodemayoediidoro,42198,#cincodemayo (EDI,I. D. Oro,en,1.0,9.781339e+12,INDEPENDENTLY PUBLISHED,NaN,YFZR,[]
19,mexicanrevolutionedioemportugusidoro,65660,#mexicanrevolution (Edição Em Português),I. D. Oro,pt-BR,1.0,NaN,INDEPENDENTLY PUBLISHED,Illuminata uma menina adolescente de dezessete...,YFZR,[]
47,cmoeligensusmascotaslosdinosaurioshowdodinosau...,66697,'cómo Eligen Sus Mascotas Los Dinosaurios? (Ho...,Jane Yolen,es,1.0,9.781338e+12,SCHOLASTIC,"""Can I keep it? It's so cute!"" In this book br...",YFP,[YFQ]
55,ngotiesekersengelafrikaanseuitgaweannaerishkigal,39693,'n Gotiese Kers Engel (Afrikaanse uitgawe),Anna Erishkigal,af,1.0,9.781943e+12,Seraphim Press,Wenner van die “eFestival of Words Best of the...,FM,[]
57,porqutenemoseldadeaccindegraciaswhywehavethank...,64831,'por Qué Tenemos El Día de Acción de Gracias=w...,Margaret Hillert,en,1.0,9.781685e+12,NORWOOD HOUSE PR,Retells the story of the Pilgrims long journey...,YF,[5HCS]


In [395]:
# same work for col_2

col_2 = prep[prep['collection'] == 2]
keys = prep['key2']
col_22 = col_2[keys.isin(keys[keys.duplicated()])]
col_22 = col_22.groupby('key2').agg({'itemID': '|'.join,
                                    'title': 'first', 
                                    'author_y': 'first',
                                    'language':'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher':'first',
                                    'description':'first',
                                    'main topic': '|'.join,
                                    'subtopics': '|'.join }).reset_index()

col_22.columns =['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']

subtopics = col_22['subtopics']

for i in range(len(col_22)):
    subtopics.at[i] = re.sub("\[|\]|\'", "", str(subtopics[i]))
    subtopics.at[i] = re.sub(",|, ", "|", str(subtopics.at[i]))
    subtopics.at[i] = str(subtopics.at[i].strip('|'))
    if subtopics.at[i] == 'nan':
        subtopics.at[i] = ''        
    
col_22['subtopics'] = subtopics

   
idTemp = np.array(col_22["itemID"].tolist())
mainTopicTemp = np.array(col_22["main topic"].tolist())
subtopicTemp = np.array(col_22["subtopics"].tolist())
idList = []
mainTopicList = []
subtopicList = []

for i in range(len(col_22)):
    arrTemp1 = idTemp[i].split('|')
    arrTemp2 = mainTopicTemp[i].split('|')
    arrTemp3 = subtopicTemp[i].split('|')
    
    idList.append(arrTemp1)
    mainTopicList.append(arrTemp2)
    subtopicList.append(arrTemp3)

# remove duplication 
for i in range(len(col_22)):
    idList[i] = list(set(idList[i]))
    mainTopicList[i] = list(set(mainTopicList[i]))
    subtopicList[i] = list(set(subtopicList[i]))
    
col_22['itemID'] = (idList)
col_22['main topic'] = (mainTopicList)
col_22['subtopics'] = (subtopicList)

# subtranct col_22 from col_2
for i in range(len(col_22)):
    for j in range(len(col_22['itemID'][i])):
        subtId = str(int(col_2[col_2['itemID'].isin([col_22['itemID'][i][j]])]['itemID']))
        idx = col_2[col_2['itemID'] == subtId].index
        col_2 = col_2.drop(idx)

col_2.rename(columns = {'key2':'key'}, inplace=True)
col_2.rename(columns = {'author_y':'author'}, inplace=True)
col_2.drop(['titleFound', 'author_x', 'key1'], axis=1, inplace=True)
col_2 = col_2[['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']]

newCol2 = pd.concat([col_2, col_22])
print(newCol2.shape)
display(newCol2.head(5))

<ipython-input-395-31e36d796e70>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  col_22 = col_2[keys.isin(keys[keys.duplicated()])]


(44393, 11)


,key,itemID,title,author,language,collection,isbn,publisher,description,main topic,subtopics
0,tripionminacciadallospaziostefanogrimaldi,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,it,2.0,9.781292e+12,Lulu.com,IL GENERE UMANO E' DESTINATO A SCOMPARIRE...QU...,FL,[]
1,theultimatevehiclecolouringbookforkidschetna,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,en,2.0,9.789390e+12,Westland Publications Limited,NaN,YBG,[]
3,4happybirthdaymallorylauriefriedman,64745,"#4 Happy Birthday, Mallory!",Laurie Friedman,en,2.0,9.781512e+12,DARBY CREEK,Audisee® eBooks with Audio combine professiona...,YF,[5HKA]
4,bastelnforfuturesusannepypke,7780,#Basteln for Future,Susanne Pypke,de,2.0,9.783736e+12,Frech Verlag GmbH,Bastle die Welt besser! Der Klimawandel geht u...,WF,"[5AF,Y]"
5,cincodemayoedicionenespaolidoro,27230,#CincoDeMayo (Edicion en español),I. D. Oro,es,2.0,NaN,INDEPENDENTLY PUBLISHED,María tiene diecisiete años y está enamorada d...,YFZR,[]


In [396]:
# same work for collection == Nan
col_nan1 = prep[prep['collection'].isnull()]
keys = prep['key1']
col_nan11 = col_nan1[keys.isin(keys[keys.duplicated()])]

col_nan11 = col_nan11.groupby('key1').agg({'itemID': '|'.join,
                                    'title': 'first', 
                                    'author_x': 'first',
                                    'language':'first',
                                    'collection':'first',
                                    'isbn':'first',
                                    'publisher':'first',
                                    'description':'first',
                                    'main topic': '|'.join,
                                    'subtopics': '|'.join }).reset_index()

col_nan11.columns =['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']


subtopics = col_nan11['subtopics']

for i in range(len(col_nan11)):
    subtopics.at[i] = re.sub("\[|\]|\'", "", str(subtopics[i]))
    
    subtopics.at[i] = re.sub(",|, ", "|", str(subtopics.at[i]))
    subtopics.at[i] = str(subtopics.at[i].strip('|'))
    if subtopics.at[i] == 'nan':
        subtopics.at[i] = ''        
    
col_nan11['subtopics'] = subtopics

idTemp = np.array(col_nan11["itemID"].tolist())
mainTopicTemp = np.array(col_nan11["main topic"].tolist())
subtopicTemp = np.array(col_nan11["subtopics"].tolist())
idList = []
mainTopicList = []
subtopicList = []

for i in range(len(col_nan11)):
    arrTemp1 = idTemp[i].split('|')
    arrTemp2 = mainTopicTemp[i].split('|')
    arrTemp3 = subtopicTemp[i].split('|')
    
    idList.append(arrTemp1)
    mainTopicList.append(arrTemp2)
    subtopicList.append(arrTemp3)
    
# remove duplication 
for i in range(len(col_nan11)):
    idList[i] = list(set(idList[i]))
    mainTopicList[i] = list(set(mainTopicList[i]))
    subtopicList[i] = list(set(subtopicList[i]))
    
col_nan11['itemID'] = (idList)
col_nan11['main topic'] = (mainTopicList)
col_nan11['subtopics'] = (subtopicList)

# subtract col_nan11 from col_nan1 
for i in range(len(col_nan11)):
    for j in range(len(col_nan11['itemID'][i])):
        subtId = str(int(col_nan1[col_nan1['itemID'].isin([col_nan11['itemID'][i][j]])]['itemID']))
        idx = col_nan1[col_nan1['itemID'] == subtId].index
        col_nan1 = col_nan1.drop(idx)

col_nan1.rename(columns = {'key1':'key'}, inplace=True)
col_nan1.rename(columns = {'author_x':'author'}, inplace=True)
col_nan1.drop(['titleFound', 'author_y', 'key2'], axis=1, inplace=True)
col_nan1 = col_nan1[['key', 'itemID', 'title', 'author', 'language', 'collection', 'isbn', 'publisher', 'description', 'main topic', 'subtopics']]

newCol3 = pd.concat([col_nan1, col_nan11])
print(newCol3.shape)
display(newCol3.head(5))

<ipython-input-396-0e4a13d24dc5>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  col_nan11 = col_nan1[keys.isin(keys[keys.duplicated()])]


(21000, 11)


,key,itemID,title,author,language,collection,isbn,publisher,description,main topic,subtopics
6,imtoothestoryofminnieandfriendswhojustlikeyout...,53261,"#ImToo - The story of Minnie and friends, who ...",Eric DeSio,NaN,NaN,NaN,Be You Books,NaN,YXQF,[YFB]
9,stickerforfuturecarmeneisendle,48408,#Sticker for Future,Carmen Eisendle,NaN,NaN,NaN,Frech Verlag GmbH,NaN,YBG,"[5AF,5HRA,5HRB,5HRC,5HRD,5JA,5JB,RNK,RNU,W,WD,..."
15,famousjillygagnon,32836,#famous,Jilly Gagnon,NaN,NaN,NaN,HarperCollins Publishers Inc,NaN,YFM,[YXQ]
18,mexicanrevolutionedizioneitalianaidoro,14300,#mexicanrevolution (Edizione Italiana),I. D. Oro,NaN,NaN,NaN,INDEPENDENTLY PUBLISHED,NaN,YFZR,[]
21,selbstschuldwasheitschonprivatthomasfeibel,73474,#selbstschuld - Was heißt schon privat,Thomas Feibel,NaN,NaN,NaN,Carlsen Verlag GmbH,NaN,JBFK4,"[5AN,JBFK4,YFB,YXHB,YXQF]"


In [397]:
# merge 

In [398]:
processedData = pd.concat([newCol1, newCol2])
processedData = pd.concat([processedData, newCol3])
processedData.reset_index(inplace=True)
processedData.drop(['key'], axis=1, inplace=True)
processedData.drop(['index'], axis=1, inplace=True)

In [399]:
processedData.shape

(72803, 10)

In [400]:
processedData.isnull().sum()

itemID             0
title              0
author             0
language       21004
collection     21000
isbn           22780
publisher          9
description    28407
main topic         0
subtopics          1
dtype: int64

In [401]:
itemIDs = processedData['itemID']
topics = processedData['main topic']
subtopics = processedData['subtopics']
             
for i in tqdm(range(len(processedData))):
    itemIDs.at[i] = re.sub("\[|\]|\'", "", str(itemIDs[i]))
    topics.at[i] = re.sub("\[|\]|\'", "", str(topics[i]))
    subtopics.at[i] = re.sub("\[|\]|\'", "", str(subtopics[i]))
    
    itemIDs.at[i] = re.sub(", ", "|", str(itemIDs[i]))
    topics.at[i] = re.sub(", ", "|", str(topics[i]))
    subtopics.at[i] = re.sub(",|, ", "|", str(subtopics[i]))

In [402]:
processedData['itemID'] = itemIDs 
processedData['main topic'] = topics
processedData['subtopics'] = subtopics

In [403]:
processedData.to_csv('../Dataset/processedData.csv')
processedData

,itemID,title,author,language,collection,isbn,publisher,description,main topic,subtopics
0,42198,#cincodemayo (EDI,I. D. Oro,en,1.0,9.781339e+12,INDEPENDENTLY PUBLISHED,NaN,YFZR,
1,65660,#mexicanrevolution (Edição Em Português),I. D. Oro,pt-BR,1.0,NaN,INDEPENDENTLY PUBLISHED,Illuminata uma menina adolescente de dezessete...,YFZR,
2,66697,'cómo Eligen Sus Mascotas Los Dinosaurios? (Ho...,Jane Yolen,es,1.0,9.781338e+12,SCHOLASTIC,"""Can I keep it? It's so cute!"" In this book br...",YFP,YFQ
3,39693,'n Gotiese Kers Engel (Afrikaanse uitgawe),Anna Erishkigal,af,1.0,9.781943e+12,Seraphim Press,Wenner van die “eFestival of Words Best of the...,FM,
4,64831,'por Qué Tenemos El Día de Acción de Gracias=w...,Margaret Hillert,en,1.0,9.781685e+12,NORWOOD HOUSE PR,Retells the story of the Pilgrims long journey...,YF,5HCS
...,...,...,...,...,...,...,...,...,...,...
72798,78942|74601,"Youth by Isaac Asimov, Science Fiction, Advent...",Isaac Asimov,NaN,NaN,NaN,Aegypan,NaN,FJ|FM,FL
72799,1824|24220,Yuánz,Tati Plath,NaN,NaN,NaN,epubli,NaN,FK,
72800,30409|16146,ZORN DER NATUR,Michel F. Bolle,NaN,NaN,NaN,tredition,NaN,FB,
72801,70346|69927,Zorn der Sonne,Harald Müller,NaN,NaN,NaN,epubli,NaN,FM,


In [404]:
processedData.to_excel('../Dataset/processedData.xlsx')

# verification

In [411]:
print(book.shape)
book.head(5)

(78030, 7)


,itemID,title,author,publisher,main topic,subtopics,key:title-author
0,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],tripionminacciadallospaziostefanogrimaldi
1,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],theultimatevehiclecolouringbookforkidschetna
2,64846,!!,,AUTHORHOUSE,FM,[],
3,64745,"#4 Happy Birthday, Mallory!",Laurie Friedman,DARBY CREEK,YF,[5HKA],4happybirthdaymallorylauriefriedman
4,7780,#Basteln for Future,"Susanne Pypke, Naturschutzjugend NAJU",Frech Verlag GmbH,WF,"[5AF,Y]",bastelnforfuturesusannepypkenaturschutzjugendnaju


In [412]:
print(processedData.shape)
processedData.tail(5)

(72803, 10)


,itemID,title,author,language,collection,isbn,publisher,description,main topic,subtopics
72798,78942|74601,"Youth by Isaac Asimov, Science Fiction, Advent...",Isaac Asimov,NaN,NaN,NaN,Aegypan,NaN,FJ|FM,FL
72799,1824|24220,Yuánz,Tati Plath,NaN,NaN,NaN,epubli,NaN,FK,
72800,30409|16146,ZORN DER NATUR,Michel F. Bolle,NaN,NaN,NaN,tredition,NaN,FB,
72801,70346|69927,Zorn der Sonne,Harald Müller,NaN,NaN,NaN,epubli,NaN,FM,
72802,6320|6092,Zwischen den Welten 01 - Daughter Of Smoke And...,Laini Taylor,NaN,NaN,NaN,FISCHER Taschenbuch,NaN,FMR,1DTJ| FYT| FBA| 1DDF-FR-LA| 1HBM| 1DDF-FR-LAA


In [413]:
apidata[apidata['titleQueried'] == "ZORN DER NATUR"]

,titleQueried,titleFound,author,language,description,isbn


In [414]:
apidata[apidata['titleQueried'] == "Zorn der Sonn"]

,titleQueried,titleFound,author,language,description,isbn


In [415]:
apidata[apidata['titleQueried'] == "1000 Gefahren auf dem Tierhof"]

,titleQueried,titleFound,author,language,description,isbn
30035,1000 Gefahren auf dem Tierhof,1000 Gefahren am Katastrophentag,Edward Packard,en,NaN,9.783474e+12
30036,1000 Gefahren auf dem Tierhof,Katastrophentag,Edward Packard,en,The reader's choices determine the course of a...,9.783474e+12
30039,1000 Gefahren auf dem Tierhof,Der Adventskalender - 1000 Gefahren in der Sta...,THiLO,de,Marie ist total aufgeregt. Der Schüleraustausc...,9.783473e+12
30040,1000 Gefahren auf dem Tierhof,Der Diamant von Burg Silberfels,Melanie Schüer,de,Als die Mutter von Minna und Heinrich krank wi...,9.783417e+12
30041,1000 Gefahren auf dem Tierhof,1000 Gefühle: Herzklopfen beim Schüleraustausch,Sonja Bullen,de,Lena freut sich wie verrückt auf ihren Schüler...,9.783473e+12
30043,1000 Gefahren auf dem Tierhof,Briefe für Alison,Sonja Bullen,de,Sophie und Alison sind beste Freundinnen. Sie ...,9.783741e+12


In [416]:
apidata[apidata['titleQueried'] == "2048"]

,titleQueried,titleFound,author,language,description,isbn
221743,2048,"H.R. 1969, H.R. 2158, H.R. 2048, and H.R. 2155",United States. Congress. House. Committee on V...,en,NaN,NaN
221744,2048,H.R. 2048 and H.RES. 30,United States. Congress. House. Committee on R...,en,NaN,NaN
221745,2048,Sitzungsberichte der Mathematisch-Naturwissens...,NaN,de,NaN,NaN
221746,2048,Producer Price Indexes,NaN,en,NaN,NaN
221747,2048,"Archiv für Ohren-, Nasen- und Kehlkopfeilkunde",NaN,de,NaN,NaN
221748,2048,Abhandlungen,Sächsische Akademie der Wissenschaften zu Leipzig,de,NaN,NaN
221749,2048,Abhandlungen der Mathematisch-Physischen Class...,NaN,de,NaN,NaN
221750,2048,Abhandlungen der Mathematisch-Physischen Klass...,NaN,de,NaN,NaN
221751,2048,Abhandlungen der Sächsischen Akademie der Wiss...,Sächsische Akademie der Wissenschaften zu Leip...,de,NaN,NaN
221752,2048,Archiv für Ohrenheilkunde,NaN,de,"""Wissenschaftliche rundschau"" in v. 1-97.",NaN


In [417]:
apidata[apidata['titleQueried'] == "Wolf"]

,titleQueried,titleFound,author,language,description,isbn
50350,Wolf,TEXT + KRITIK 215 - Wolf Wondratschek,"Hans-Edwin Friedrich, Christoph Rauen",de,NaN,9.783869e+12
50352,Wolf,Vorlesungen über die Alterthumswissenschaft,Friedrich August Wolf,de,NaN,NaN
50353,Wolf,Der mit seinen Tugenden Triumphierende Adel .....,Wolf Max Anton Graf von Ueberacker,de,NaN,NaN
50354,Wolf,Die Orlandi-Verschwörung,Martin de Wolf,de,NaN,9.783837e+12
50355,Wolf,Frankreich oh làlà,Wolf Buchinger,de,Dieses Buch ist eine spezielle Liebeserklärung...,9.783750e+12
50356,Wolf,Psychologische Analyse und theologische Wahrheit,Wolf-Ulrich Klünker,de,NaN,9.783526e+12
50357,Wolf,Wolf,NaN,en,NaN,NaN
50358,Wolf,Colorado Gray Wolf Recovery,Larry E. Bennett,en,NaN,NaN
50360,Wolf,TEXT + KRITIK 215 - Wolf Wondratschek,"Hans-Edwin Friedrich, Christoph Rauen",de,NaN,9.783869e+12
50362,Wolf,Vorlesungen über die Alterthumswissenschaft,Friedrich August Wolf,de,NaN,NaN
